In [1]:
import sys
from typing import *
from dataclasses import *
sys.version

'3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]'

In [2]:
# without @dataclass

class Fraction:
    numerator: int = 0
    denominator: int = 1

# f = Fraction(1, 2)                       # TypeError: Fraction() takes no arguments
# f = Fraction(numerator=1, denominator=2) # TypeError: Fraction() takes no arguments
f = Fraction()
f.numerator = 1
f.denominator = 2
print(f)
g = Fraction()
g.numerator = 1
g.denominator = 2
print(f == g) # False

False


In [3]:
@dataclass
class Fraction:
    numerator: int = 0
    denominator: int = 1

f = Fraction(1, 2)                       # okay
f = Fraction(numerator=1, denominator=2) # okay
print(f)
g = Fraction()
g.numerator = 1
g.denominator = 2
print(f == g)   # True
# print(f < g)  # TypeError: '<' not supported between instances of 'Fraction' and 'Fraction'
                # for this to "work", we have to pass `order = True` to @dataclass

Fraction(numerator=1, denominator=2)
True


In [4]:
@dataclass(order=True)
class Fraction:
    numerator: int = 0
    denominator: int = 1

f = Fraction(1, 1)
g = Fraction(3, 6)
print(f < g) # True, because f.numerator < g.numerator
             # the default operators just compare attribute-by-attribute
             # in the order the attributes were declared in the class

True


In [5]:
@dataclass(frozen=True)
class Fraction:
    numerator: int = 0
    denominator: int = 1

f = Fraction(1, 2)
# f.numerator = 2 # FrozenInstanceError: cannot assign to field 'numerator'

In [6]:
@dataclass
class Fraction:
    numerator: int = 0
    denominator: int = 1

f = Fraction(1, 2)
print(f.numerator)
# print(f['numerator'])       # TypeError: 'Fraction' object is not subscriptable
print(asdict(f))              # {'numerator': 1, 'denominator': 2}
print(astuple(f))             # (1, 2)

1
{'numerator': 1, 'denominator': 2}
(1, 2)


In [7]:
class Fraction:
    numerator: int = 0
    denominator: int = 1
    def mul(self, x: int):
        self.numerator *= x

print(Fraction.__dict__)
print()
print(dir(Fraction))
print()
print(Fraction.__annotations__)

{'__module__': '__main__', '__annotations__': {'numerator': <class 'int'>, 'denominator': <class 'int'>}, 'numerator': 0, 'denominator': 1, 'mul': <function Fraction.mul at 0x000002631152AD30>, '__dict__': <attribute '__dict__' of 'Fraction' objects>, '__weakref__': <attribute '__weakref__' of 'Fraction' objects>, '__doc__': None}

['__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'denominator', 'mul', 'numerator']

{'numerator': <class 'int'>, 'denominator': <class 'int'>}


In [8]:
def asdict(x):
    return {a:getattr(x, a) for a in x.__class__._attribs}

def astuple(x):
    return tuple(getattr(x, a) for a in x.__class__._attribs)

def dataclass(cls=None, **kwargs):
    def decorator(cls):
        cls._attribs = cls.__annotations__.keys()
        # define initializer, unless defined by the user
        if '__init__' not in cls.__dict__:
            def __init__(self, *args, **kwargs):
                if len(args) > 0:
                    for attrib, arg in zip(self.__class__._attribs, args):
                        # avoid our own __setattr__ in case it's a frozen dataclass:
                        object.__setattr__(self, attrib, arg)                       
                elif len(kwargs) > 0:
                    for attrib in self.__class__._attribs:
                        # avoid our own __setattr__ in case it's a frozen dataclass:
                        object.__setattr__(self, attrib, kwargs[attrib])
            cls.__init__ = __init__
        # define string conversion, unless defined by th user
        if '__str__' not in cls.__dict__:
            def __str__(self):
                kv_tuples = [(attrib, getattr(self, attrib)) for attrib in self.__class__._attribs]
                kv_str = ', '.join([f'{k}={v}' for (k, v) in kv_tuples])
                return f'{self.__class__.__name__}({kv_str})'
            cls.__str__ = __str__
        if kwargs.get('eq', True):
            # define ==, unless defined by the user
            if '__eq__' not in cls.__dict__:
                def __eq__(self, other):
                    for attrib in self.__class__._attribs:
                        if getattr(self, attrib) != getattr(other, attrib):
                            return False
                    return True            
                cls.__eq__ = __eq__
        if kwargs.get('order', False):
            # define <, <=, >, >=, unless defined by the user
            if '__lt__' not in cls.__dict__:
                def __lt__(self, other):
                    for attrib in self.__class__._attribs:
                        if getattr(self, attrib) >= getattr(other, attrib):
                            return False
                    return True            
                cls.__lt__ = __lt__
            if '__le__' not in cls.__dict__:
                def __le__(self, other):
                    for attrib in self.__class__._attribs:
                        if getattr(self, attrib) > getattr(other, attrib):
                            return False
                    return True            
                cls.__le__ = __le__
            if '__gt__' not in cls.__dict__:
                def __gt__(self, other):
                    for attrib in self.__class__._attribs:
                        if getattr(self, attrib) <= getattr(other, attrib):
                            return False
                    return True
                cls.__gt__ = __gt__
            if '__ge__' not in cls.__dict__:
                def __ge__(self, other):
                    for attrib in self.__class__._attribs:
                        if getattr(self, attrib) < getattr(other, attrib):
                            return False
                    return True
                cls.__ge__ = __ge__
        if kwargs.get('frozen', False):
            # don't allow changing attributes
            def __setattr__(self, attrib, value):
                if attrib not in self.__class__._attribs:
                    setattr(self, attrib, value)
                else:
                    raise AttributeError(f'dataclass is frozen, cannot assign to field \'{attrib}\'')
            cls.__setattr__ = __setattr__
        return cls
    if cls is None:
        # decorator was used like @dataclass(...)
        return decorator
    else:
        # decorator was used like @dataclass, without parens
        return decorator(cls)

@dataclass(frozen=True)
class Fraction:
    numerator: int = 0
    denominator: int = 1
    def mul(self, x: int):
        self.numerator *= x

f = Fraction(numerator=1, denominator=2)
print(asdict(f))
print(astuple(f))
print(f)
# f.numerator = 1
g = Fraction(numerator=1, denominator=2)
print(f==g)
# f.numerator = 3
#Fraction(numerator=1, denominator=2)<Fraction(numerator=2, denominator=3)

{'numerator': 1, 'denominator': 2}
(1, 2)
Fraction(numerator=1, denominator=2)
True
